## EDA

1. Setup + imports

In [220]:
from pathlib import Path
import sys
import pandas as pd
import os
import plotly.express as px

In [221]:
ROOT = Path(os.path.abspath('')).resolve().parent
sys.path.append(str(ROOT / "src"))

In [222]:
from bootcamp_data.config import make_paths
from bootcamp_data.io import read_parquet
from bootcamp_data.transforms import missingness_report

In [223]:
paths =make_paths(ROOT)

2. Load processed data


In [224]:
df=read_parquet(paths.Processed/'analytics_table.parquet')

3. Quick audit

In [225]:
df

,order_id,user_id,amount,quantity,created_at,status,status_clean,amount__isna,quantity__isna,date,year,month,dow,hour,country,signup_date,amount_winsor,amount__is_outlier
0,A0001,0001,12.5,1,2025-12-01 10:05:00+00:00,Paid,paid,False,False,2025-12-01,2025.0,2025-12,Monday,10.0,SA,2025-11-15,12.5,False
1,A0002,0002,8.0,2,2025-12-01 11:10:00+00:00,paid,paid,False,False,2025-12-01,2025.0,2025-12,Monday,11.0,SA,2025-11-20,8.0,False
2,A0003,0003,<NA>,1,2025-12-02 09:00:00+00:00,Refund,refund,True,False,2025-12-02,2025.0,2025-12,Tuesday,9.0,AE,2025-11-22,<NA>,<NA>
3,A0004,0001,25.0,<NA>,2025-12-03 14:30:00+00:00,PAID,paid,False,True,2025-12-03,2025.0,2025-12,Wednesday,14.0,SA,2025-11-15,25.0,False
4,A0005,0004,100.0,1,NaT,paid,paid,False,False,None,NaN,<NA>,None,NaN,SA,2025-11-25,100.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,A0096,0069,18.5,1,2026-01-19 10:00:00+00:00,paid,paid,False,False,2026-01-19,2026.0,2026-01,Monday,10.0,SA,2025-11-24,18.5,False
96,A0097,0070,30.0,2,2026-01-19 14:45:00+00:00,Paid,paid,False,False,2026-01-19,2026.0,2026-01,Monday,14.0,SA,2025-11-25,30.0,False
97,A0098,0002,7.0,<NA>,2026-01-20 08:15:00+00:00,PAID,paid,False,True,2026-01-20,2026.0,2026-01,Tuesday,8.0,SA,2025-11-20,7.0,False
98,A0099,0071,55.0,1,2026-01-20 12:00:00+00:00,paid,paid,False,False,2026-01-20,2026.0,2026-01,Tuesday,12.0,AE,2025-11-26,55.0,False


In [226]:
#row count
df.count()

order_id              100
user_id               100
amount                 88
quantity               92
created_at             93
status                100
status_clean          100
amount__isna          100
quantity__isna        100
date                   93
year                   93
month                  93
dow                    93
hour                   93
country               100
signup_date           100
amount_winsor          88
amount__is_outlier     88
dtype: int64

In [227]:
#dtypes (first 15)
df.dtypes[:16]

order_id               string[python]
user_id                string[python]
amount                        Float64
quantity                        Int64
created_at        datetime64[ns, UTC]
status                         object
status_clean                   object
amount__isna                     bool
quantity__isna                   bool
date                           object
year                          float64
month                  string[python]
dow                            object
hour                          float64
country                        object
signup_date                    object
dtype: object

In [228]:
#top missing columns
missingness_report(df).head()

,n_missing,p_missing
amount_winsor,12,0.12
amount,12,0.12
amount__is_outlier,12,0.12
quantity,8,0.08
hour,7,0.07


4. Questions + results


##### Questions
1. what is the highest quantity for a country? 

2. what is each country's revenue?

3. what is the Revenue over time?

4. what is the Amount distribution?


#1 what is the highest quantity for a country? 

In [244]:
data_chart=df.groupby("country").agg(quantity=("quantity","sum")).reset_index().sort_values("quantity", ascending=False)
chart_2=px.bar(data_chart,x='country',y='quantity',title="Quantity by Country")
chart_2.show()

In [245]:
FIGS = ROOT/"reports"/"figures"
FIGS.mkdir(parents=True, exist_ok=True)

def save_fig(fig, path: Path, *, scale: int = 2) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(path), scale=scale)

In [246]:
save_fig(chart_2,FIGS/"quantity_by_country.png")

#2 what is each country's revenue?

In [247]:
rev = (
df.groupby("country", dropna=False)
.agg(
n=("order_id","size"),
revenue=("amount","sum"),
aov=("amount","mean"),
)
.reset_index()
.sort_values("revenue", ascending=False)
)

In [248]:
rev

,country,n,revenue,aov
1,SA,76,2753.23,40.488676
0,AE,24,649.48,32.474


In [249]:
fig = px.bar(rev, x="country", y="revenue", title="Revenue by country (all data)")
fig.update_layout(title={"x": 0.02})
fig = px.bar(rev, x="country", y="revenue", title="Revenue by country (all data)")
fig.update_xaxes(title_text="Country")
fig.update_yaxes(title_text="Revenue (sum of amount)")
save_fig(fig, FIGS / "revenue_by_country.png")

In [250]:
fig

#3 what is the Revenue over time?

In [251]:

trend = (
    df.groupby("date", dropna=False)
    .agg(n=("order_id","size"), revenue=("amount","sum"))
    .reset_index()
    .sort_values("date")
)

trend.head()

,date,n,revenue
0,2025-12-01,2,20.5
1,2025-12-02,1,0.0
2,2025-12-03,1,25.0
3,2025-12-04,2,60.75
4,2025-12-05,1,0.0


In [252]:
fig.update_layout(title={"x": 0.02})
fig = px.line(trend, x="date", y="revenue", title="Revenue over time")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Revenue")
save_fig(fig, FIGS / "revenue_trend.png")

fig

#4 what is the Amount distribution?

In [253]:
fig = px.histogram(df, x="amount_winsor", nbins=30, title="Order amount disturbutin")
fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Amount (winsorized)")
fig.update_yaxes(title_text="Number of orders")
save_fig(fig, FIGS / "amount_hist_winsor.png")
fig

5. Bootstrap comparison

In [254]:
import numpy as np

In [255]:
def bootstrap_diff_means(a: pd.Series, b: pd.Series, *, n_boot: int = 2000, seed: int = 0) -> dict:
    rng = np.random.default_rng(seed)
    a = pd.to_numeric(a, errors="coerce").dropna().to_numpy()
    b = pd.to_numeric(b, errors="coerce").dropna().to_numpy()
    assert len(a) > 0 and len(b) > 0, "Empty group after cleaning"

    diffs = []
    for _ in range(n_boot):
        sa = rng.choice(a, size=len(a), replace=True)
        sb = rng.choice(b, size=len(b), replace=True)
        diffs.append(sa.mean() - sb.mean())
    diffs = np.array(diffs)

    return {
    "diff_mean": float(a.mean() - b.mean()),
    "ci_low": float(np.quantile(diffs, 0.025)),
    "ci_high": float(np.quantile(diffs, 0.975)),
    }

In [256]:
d = df.assign(is_refund=df["status_clean"].eq("refund").astype(int))
d

,order_id,user_id,amount,quantity,created_at,status,status_clean,amount__isna,quantity__isna,date,year,month,dow,hour,country,signup_date,amount_winsor,amount__is_outlier,is_refund
0,A0001,0001,12.5,1,2025-12-01 10:05:00+00:00,Paid,paid,False,False,2025-12-01,2025.0,2025-12,Monday,10.0,SA,2025-11-15,12.5,False,0
1,A0002,0002,8.0,2,2025-12-01 11:10:00+00:00,paid,paid,False,False,2025-12-01,2025.0,2025-12,Monday,11.0,SA,2025-11-20,8.0,False,0
2,A0003,0003,<NA>,1,2025-12-02 09:00:00+00:00,Refund,refund,True,False,2025-12-02,2025.0,2025-12,Tuesday,9.0,AE,2025-11-22,<NA>,<NA>,1
3,A0004,0001,25.0,<NA>,2025-12-03 14:30:00+00:00,PAID,paid,False,True,2025-12-03,2025.0,2025-12,Wednesday,14.0,SA,2025-11-15,25.0,False,0
4,A0005,0004,100.0,1,NaT,paid,paid,False,False,None,NaN,<NA>,None,NaN,SA,2025-11-25,100.0,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,A0096,0069,18.5,1,2026-01-19 10:00:00+00:00,paid,paid,False,False,2026-01-19,2026.0,2026-01,Monday,10.0,SA,2025-11-24,18.5,False,0
96,A0097,0070,30.0,2,2026-01-19 14:45:00+00:00,Paid,paid,False,False,2026-01-19,2026.0,2026-01,Monday,14.0,SA,2025-11-25,30.0,False,0
97,A0098,0002,7.0,<NA>,2026-01-20 08:15:00+00:00,PAID,paid,False,True,2026-01-20,2026.0,2026-01,Tuesday,8.0,SA,2025-11-20,7.0,False,0
98,A0099,0071,55.0,1,2026-01-20 12:00:00+00:00,paid,paid,False,False,2026-01-20,2026.0,2026-01,Tuesday,12.0,AE,2025-11-26,55.0,False,0


In [257]:
a = d.loc[d["country"].eq("SA"), "is_refund"]
b = d.loc[d["country"].eq("AE"), "is_refund"]

In [258]:
bootstrap_diff_means(a,b)

{'diff_mean': -0.0899122807017544,
 'ci_low': -0.2807017543859649,
 'ci_high': 0.07461622807017514}

6. Findings + caveats

1- the highest quantity for a country is 108 by Saudi Arabia (SA) which is much higher that Arab Emarites's (AE) quantity which is 27.

2- the total revenues for Saudi Arabia and Arab Emarites are 2753.23 and 649.48 respectively.

3- the Revenue peakes each week towards the end of the week, and the Revenues tend to get higher at the begining of eac month and lower at the end.

4- the data shows that the lesser the amounts the higher the number of orders.
